################Topic generation:::Preprocess#################################################

In [126]:
#Get the data
import importlib
import pandas as pd
google_data = pd.read_csv("data"+os.sep+"part"+os.sep+"Suomi112_eng_google.csv", encoding='utf-8-sig')['content']
apple_data = pd.read_csv("data"+os.sep+"part"+os.sep+"Suomi112_eng_apple.csv", encoding='utf-8-sig')['content']

In [162]:
data = pd.concat([google_data,apple_data])

In [128]:
import utils
importlib.reload(utils)
processor = utils.Processor()
processor.ini_dowload()

In [163]:
data = data.apply(lambda x: processor.preprocess(x))
data = data.apply(lambda x: processor.tokenize(x))
data = data.apply(lambda x: processor.remove_stopwords(x))
data = data.apply(lambda x: processor.process_tokens(x))
data
data_copy = data

################Topic generation:::Topics#################################################
Mostly taken from https://towardsdatascience.com/topic-modeling-and-latent-dirichlet-allocation-in-python-9bf156893c24

In [172]:
flat_data = [tag for sentence in data for tag in sentence ]
from gensim.corpora import Dictionary
dictionary = Dictionary(data)
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)
bow_corpus = [dictionary.doc2bow(doc) for doc in data]

In [169]:
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1

0 ask
1 call
2 fortun
3 know
4 number
5 phone
6 open
7 screen
8 posit
9 thumb
10 address
11 need
12 real
13 situat
14 use
15 good
16 chang
17 delet
18 emerg
19 set
20 thing
21 time
22 applic
23 case
24 import
25 instal
26 work
27 could
28 great
29 idea
30 rememb
31 still
32 would
33 find
34 lose
35 necessari
36 realli
37 automat
38 also
39 center
40 inform
41 coordin
42 locat
43 send
44 crash
45 danger
46 even
47 live
48 make
49 notif
50 peopl
51 think
52 though
53 well
54 data
55 face
56 alreadi
57 first
58 everi
59 cannot
60 android
61 help
62 notic
63 person
64 servic
65 easi
66 announc
67 constant
68 go
69 option
70 otherwis
71 traffic
72 turn
73 life
74 definit
75 mani
76 worth
77 recommend
78 safe
79 alway
80 happen
81 someth
82 quick
83 experi
84 like
85 remov
86 seem
87 take
88 download
89 updat
90 give
91 save
92 possibl
93 probabl
94 quit
95 nice
96 button
97 come
98 immedi
99 thank
100 useless
101 user
102 start
103 tri
104 everyon
105 addit
106 especi
107 show
108 exampl
10

In [182]:
bow_doc_4310 = bow_corpus[1]
for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0],dictionary[bow_doc_4310[i][0]],bow_doc_4310[i][1]))

Word 5 ("phone") appears 1 time.
Word 6 ("open") appears 1 time.
Word 7 ("screen") appears 1 time.


In [177]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.40444358736290076),
 (1, 0.46135364385744315),
 (2, 0.45487355340142943),
 (3, 0.2984760017100485),
 (4, 0.5327929747022492),
 (5, 0.2090740846388082)]


In [178]:
import gensim
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.090*"locat" + 0.070*"good" + 0.048*"know" + 0.042*"applic" + 0.033*"call" + 0.030*"help" + 0.029*"use" + 0.023*"realli" + 0.023*"phone" + 0.022*"safe"
Topic: 1 
Words: 0.093*"import" + 0.080*"applic" + 0.049*"good" + 0.038*"would" + 0.035*"otherwis" + 0.034*"posit" + 0.024*"everi" + 0.020*"come" + 0.019*"servic" + 0.019*"phone"
Topic: 2 
Words: 0.063*"number" + 0.059*"good" + 0.053*"realli" + 0.044*"emerg" + 0.043*"applic" + 0.043*"would" + 0.037*"call" + 0.034*"phone" + 0.029*"could" + 0.023*"crash"
Topic: 3 
Words: 0.075*"work" + 0.071*"phone" + 0.050*"applic" + 0.035*"open" + 0.029*"number" + 0.028*"great" + 0.025*"download" + 0.025*"locat" + 0.025*"oneplus" + 0.022*"call"
Topic: 4 
Words: 0.077*"necessari" + 0.072*"call" + 0.055*"emerg" + 0.053*"inform" + 0.050*"center" + 0.048*"locat" + 0.046*"applic" + 0.035*"also" + 0.026*"good" + 0.025*"fortun"
Topic: 5 
Words: 0.065*"batteri" + 0.051*"phone" + 0.048*"emerg" + 0.045*"applic" + 0.038*"locat" + 0.036*"call" + 0

In [180]:
import gensim
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.056*"phone" + 0.051*"number" + 0.040*"thing" + 0.036*"time" + 0.028*"good" + 0.028*"open" + 0.026*"applic" + 0.024*"screen" + 0.023*"tri" + 0.021*"call"
Topic: 1 Word: 0.042*"good" + 0.041*"batteri" + 0.032*"posit" + 0.032*"applic" + 0.031*"background" + 0.030*"locat" + 0.030*"worth" + 0.025*"like" + 0.024*"hour" + 0.019*"use"
Topic: 2 Word: 0.072*"import" + 0.038*"announc" + 0.038*"well" + 0.030*"immedi" + 0.029*"even" + 0.027*"experi" + 0.025*"servic" + 0.023*"quit" + 0.023*"work" + 0.022*"coordin"
Topic: 3 Word: 0.341*"good" + 0.070*"great" + 0.032*"thumb" + 0.029*"idea" + 0.020*"applic" + 0.020*"hope" + 0.017*"inform" + 0.014*"everyon" + 0.013*"away" + 0.013*"someth"
Topic: 4 Word: 0.054*"know" + 0.051*"emerg" + 0.049*"good" + 0.048*"need" + 0.036*"place" + 0.033*"help" + 0.026*"thank" + 0.025*"find" + 0.023*"locat" + 0.021*"realli"
Topic: 5 Word: 0.065*"download" + 0.043*"applic" + 0.040*"import" + 0.030*"find" + 0.027*"practic" + 0.024*"instal" + 0.023*"call" + 0

In [183]:
for index, score in sorted(lda_model_tfidf[bow_corpus[1]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.7749852538108826	 
Topic: 0.056*"phone" + 0.051*"number" + 0.040*"thing" + 0.036*"time" + 0.028*"good" + 0.028*"open" + 0.026*"applic" + 0.024*"screen" + 0.023*"tri" + 0.021*"call"

Score: 0.02500375360250473	 
Topic: 0.041*"save" + 0.033*"emerg" + 0.032*"life" + 0.030*"version" + 0.029*"number" + 0.029*"phone" + 0.028*"center" + 0.026*"call" + 0.026*"locat" + 0.026*"right"

Score: 0.02500278875231743	 
Topic: 0.086*"work" + 0.081*"necessari" + 0.056*"safe" + 0.046*"good" + 0.046*"oneplus" + 0.032*"phone" + 0.030*"crash" + 0.027*"updat" + 0.024*"come" + 0.022*"applic"

Score: 0.025001896545290947	 
Topic: 0.134*"use" + 0.086*"realli" + 0.047*"everyon" + 0.027*"call" + 0.025*"applic" + 0.023*"data" + 0.021*"import" + 0.019*"emerg" + 0.018*"well" + 0.016*"work"

Score: 0.02500154636800289	 
Topic: 0.065*"download" + 0.043*"applic" + 0.040*"import" + 0.030*"find" + 0.027*"practic" + 0.024*"instal" + 0.023*"call" + 0.022*"phone" + 0.018*"connect" + 0.018*"good"

Score: 0.02500152